In [34]:
import numpy as np
import matplotlib.pyplot as plt
import difflib
import csv
import turicreate as tc
import skfuzzy as fuzz
import numpy_indexed as npi
import pyamg

from turicreate import SFrame, SArray
from math import sin, cos, sqrt, atan2, radians
from scipy.spatial import distance
from astropy.table import Table
from astropy.io.ascii import write
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from scipy.stats import entropy
from skcmeans.algorithms import CMeans
from skcmeans.algorithms import Probabilistic
from skcmeans.algorithms import Fuzzy
from pcm.cmeans import pcm

from neupy import algorithms, environment

from html.parser import HTMLParser

import urllib.request as urllib2

In [102]:
class MyHTMLParser(HTMLParser):

    recording = False
    gps = list()

    def handle_starttag(self, tag, attrs):
        if tag == 'lon' or tag == 'lat':
            self.recording = True

    def handle_endtag(self, tag):
        if tag == 'lon' or tag == 'lat':
            self.recording = False

    def handle_data(self, data):
        if self.recording:
            self.gps.append(data)


In [106]:
list_gps = []
count = 0
with open('trip_detail_new.csv') as file:
    reader = csv.reader(file)
    for i in reader:
        if count != 0:
            parser = MyHTMLParser()
            parser.gps = list()
            stop_id = str(i[2])
            html = 'http://api.pugetsound.onebusaway.org/api/where/stop/'+stop_id+'.xml?key=aef735ee-d585-445e-9ab4-dcba2a85804f'
            html_page = urllib2.urlopen(html)
            parser.feed(str(html_page.read()))
            list_gps.append(parser.gps)
        count+=1


In [107]:
myFile = open('info.csv', 'w')  
with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(list_gps)

In [6]:
def returnShorter(a, b):
    if len(a) > len(b):
        return b
    else:
        return a
    
def load_data(input_csv, gps):
    line = []
    for file in input_csv:
        with open(file) as csvfile:
            reader = csv.reader(csvfile, delimiter=' ')
            for row in reader:
                if ';' in row[1]:
                    row[1] = row[1].split(';')
                else:
                    row[0] = format_transform(row[0])
                    row[1] = row[1].split(',')
                if '' in row[1]:
                    row[1].remove('')
                row[1].insert(0, row[0])
                del(row[1][2:4])
                if(gps):
                    del(row[1][4:])
                else:
                    del(row[1][3:])
                line.append(row[1])
    return line
    
# Create SFrame for gps data and sensing data
def create_frame(list_data, name, time_frame):
    list_date = []
    list_time = []
    if 'gps' in name:
        list_lat = []
        list_lon = []
        for data in list_data:
        # Iterate thourgh the whole list of gps_data and extract values accordingly
            list_date.append(data[0])
            list_time.append(data[1])
            list_lat.append(data[2])
            list_lon.append(data[3])
        return SFrame({'date':list_date, 'time':list_time, 'lat': list_lat, 'lon':list_lon})
    else:
        list_mac = []
        for data in list_data:
            time = data[1]
            date = data[0]
            in_range = False
            for item in time_frame:
                if date == item['date'] and time_difference(item['start_time'], time) > -120 and time_difference(time, item['end_time']) > -240:
                    in_range = True
            # Only those data within the range of trip
            if in_range:
            # Iterate thourgh the whole list of gps_data and extract values accordingly
                list_date.append(data[0])
                list_time.append(data[1])
                list_mac.append(data[2])
        return SFrame({'date':list_date, 'time':list_time, 'mac':list_mac})
    
def check_error(count, mac, date):
    amount = 0
    for item in sen_frame:
        if item['mac'] == mac and item['date'] == date:
            amount+=1
    if amount != count:
        print(mac, date)

def check_last_index(a, element):
    index = -1
    position = 0
    for item in a:
        if item == element:
            index = position
        position+=1
    return index

def time_convert(time):
    s_list = time.split(':')
    if len(s_list) == 3:
        return 3600*float(s_list[0]) + 60*float(s_list[1]) + float(s_list[2])
    else:
        return 3600*float(s_list[0]) + 60*float(s_list[1])

def time_difference(time1, time2):
    return time_convert(time2) - time_convert(time1)

def find(tag1, con1, tag2, con2):
    for item in data:
        if item[tag1] == con1 and item[tag2] == con2:
            print(item)

def find_distance(s_lat, s_lon, e_lat, e_lon):
    R = 6373.0

    lat1 = radians(float(s_lat))
    lon1 = radians(float(s_lon))
    lat2 = radians(float(e_lat))
    lon2 = radians(float(e_lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 1000

def compute_distance(set1, set2):
    cumulative = 0
    summation = 0
    for i, j in zip(set1, set2):
        cumulative += (i-j)**2
        if j != 0:
            summation += i/j
        else:
            summation += i/1
    return sqrt(cumulative)

def normalize(v):
    largest = v.max()
    if largest == 0: 
        return v
    return v / largest

# Compute the Silhouette Coefficient
def compute_Sil_socre(data, labels):
    score = metrics.silhouette_score(data, labels, metric='euclidean')
    return score
    
def format_transform(date):
    # temp[0]: Month temp[1]: Day temp[2]: Year
    temp = date.split('/')
    if len(temp[1]) == 2 and len(temp[0]) == 1:
        return '20' + temp[2] + '-0' + temp[0] + '-' + temp[1]
    elif len(temp[1]) == 1 and len(temp[0]) == 2:
        return '20' + temp[2] + '-' + temp[0] + '-0' + temp[1]
    elif len(temp[1]) == 2 and len(temp[0]) == 2:
        return '20' + temp[2] + '-' + temp[0] + '-' + temp[1]
    else:
        return '20' + temp[2] + '-0' + temp[0] + '-0' + temp[1]

def computePassen(labels):  
    count = 0
    for i in range(len(labels)):
        if labels[i] == 1:
            count+=1
    print(count)

def evaluate(a, b):
    score_mape = mape(a, b)
    score_mse = mse(a, b)
    score_smape = smape(a, b)
    print("Mean Abosulute Percentage Error: ", score_mape)
    print("Mean Square Error: ", score_mse)
    print("Symmetry Mean Abosulute Percentage Error: ", score_smape)

def mape(a, b): 
    mask = a != 0
    return (np.fabs(a - b)/a)[mask].mean()

def mse(a, b):
    return mean_squared_error(a, b)

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def multiply(a, scalor):
    for i in range(len(a)):
        a[i] = int(round(a[i] * scalor))
    return a

def list_difference(list1, list2):
    return list(set(list1) - set(list2))

def setNone(result):
    result_none = [[0 for a in range(len(result))] for b in range(len(result))]
    for i in range(len(result)):
        for j in range(len(result)):
            if j < i:
                result_none[i][j] = None
            else:
                result_none[i][j] = result[i][j]
    return result_none

def Relu(result):
    for i in range(len(result)):
        result[i] = result[i].apply(lambda x: round(x) if x > 0 else 0)
    return result

def returnMin(time_frame, time, date):
    minimum = 100000000
    index = 0
    count = 0
    for i in time_frame:
        # Compute the distance between two times
        if date == i['date']:
            difference = abs(time_convert(i['time']) - time_convert(time))
            if difference < minimum:
                minimum = difference
                index = count
        count+=1
    return index, (time_frame[index]['passenger'] == 0)

def weight_validation(data, label):
    # The lower the better
    # Search for labels(clusters)
    indices = []
    for i in range(2):
        indices.append(np.where(label == i)[0])
    # indices has two clusters labeled 0 and 1
    # Calculate the intra-cluster weights(Win)
    intra_weight = 0
    for i in range(2):
        for j in indices[i]:
            for k in indices[i]:
                intra_weight += compute_distance(data[j], data[k])
    intra_weight = intra_weight/2
    
    # Compute the inter-cluster weights(Wout)
    inter_weight = 0
    for i in indices[0]:
        for j in indices[1]:
            inter_weight += compute_distance(data[i], data[j])
    
    # Compute the number of distinct intra-cluster edges(Nin)
    intra_num = 0
    for i in range(2):
        intra_num += len(indices[i]) * (len(indices[i]) - 1) / 2 
    
    # Compute the number of distinct inter-cluster edges(Nout)
    inter_num = len(indices[0]) * len(indices[1])
    
    # BetaCV: smaller the BetaCV ratio, the better the clustering.
    BetaCV = (intra_weight / intra_num) / (inter_weight / inter_num)
    # Nomalized Cut: The higher the normalized cut value the better.
    Norm_index = inter_weight / (intra_weight + inter_weight)
    
    # BetaCV measure
    return BetaCV, Norm_index

def Dunn_index(data, label):
    # The larger the better
    # Search for labels(clusters)
    indices = []
    Win = []
    Wout = []
    for i in range(2):
        indices.append(np.where(label == i)[0])
    # Construct a list which are all the intra-cluster distance
    for i in range(2):
        for j in indices[i]:
            for k in indices[i]:
                Win.append(compute_distance(data[j], data[k]))
    # Construct a list which are all the inter-cluster distance
    for i in indices[0]:
        for j in indices[1]:
            Wout.append(compute_distance(data[i], data[j]))
    # Find Dunn Index
    return min(Wout)/max(Win)

def DaviesBouldin(data, label):
    # Smaller is better
    n_cluster = len(np.bincount(label))
    cluster_k = [data[label == k] for k in range(n_cluster)]
    centroids = [np.mean(k, axis = 0) for k in cluster_k]
    variances = [np.mean([compute_distance(p, centroids[i]) for p in k]) for i, k in enumerate(cluster_k)]
    db = []

    for i in range(n_cluster):
        for j in range(n_cluster):
            if j != i:
                db.append((variances[i] + variances[j]) / compute_distance(centroids[i], centroids[j]))

    return(np.max(db) / n_cluster)
    

In [1]:
gps_cat = ['ts', 'sensorid', 'gpslat', 'gpslon']
gps_sen = ['ts', 'sensorid', 'mac']
csv_gps = ['mov_gps_1.csv', 'mov_gps_2.csv']
csv_sen = ['mov_sensing_1.csv', 'mov_sensing_2.csv']

In [7]:
# Make a frame which contains the data for each trip
with open('trip_detail_new.csv') as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    start_time = []
    end_time = []
    trip_list = []
    date_list = []
    start = ''
    end = ''
    for row in reader:
        if count != 0:
            trip = row[0]
            start = row[5].split(' ')[1]
            date = format_transform(row[5].split(' ')[0])
            if trip not in trip_list:
                trip_list.append(trip)
                start_time.append(start)
                date_list.append(date)
                if end != '':
                    end_time.append(end)
            end = start
        count+=1
    end_time.append(end)
time_frame = SFrame({'start_time': start_time, 'end_time':end_time, 'trip_id': trip_list, 'date': date_list})

In [8]:
gps_data = load_data(csv_gps, True)
sen_data = load_data(csv_sen, False)

In [12]:
def time_difference(time1, time2):
    return time_convert(time2) - time_convert(time1)

def time_convert(time):
    s_list = time.split(':')
    if len(s_list) == 3:
        return 3600*float(s_list[0]) + 60*float(s_list[1]) + float(s_list[2])
    else:
        return 3600*float(s_list[0]) + 60*float(s_list[1])

In [13]:
gps_frame = create_frame(gps_data, 'gps', time_frame)
sen_frame = create_frame(sen_data, 'sensing', time_frame)
gps_frame = gps_frame.unique()
gps_frame = gps_frame.sort(['date', 'time'])
sen_frame = sen_frame.unique()
sen_frame = sen_frame.sort(['date', 'time'])

In [14]:
# Change the format of the time
list_time = []
for i in range(len(sen_frame)):
    time = sen_frame[i]['time'].split(':')
    time[2] = round(float(time[2]))
    if time[2] >= 60:
        time[1] = int(time[1]) + 1
        time[2] -= 60
    if int(time[1]) >= 60:
        time[0] = int(time[0]) + 1
        time[1] -= 60
    time[0] = str(time[0])
    time[1] = str(time[1])
    time[2] = str(time[2])
    if len(time[2]) == 1:
        time[2] = '0' + time[2]
    if len(time[1]) == 1:
        time[1] = '0' + time[1]
    if len(time[0]) == 1:
        time[0] = '0' + time[0]
    time = ''+time[0]+':'+time[1]+':'+str(time[2])
    list_time.append(time)
temp = SFrame({'time': list_time})
sen_frame = sen_frame.remove_columns(['time'])
sen_frame = sen_frame.add_columns(temp)

In [ ]:
# Associate the GPS location with each MAC according to the time
list_lon = []
list_lat = []
for element in sen_frame:
    date = element['date']
    time = element['time']
    lon = ''
    lat = ''
    for i in gps_frame:
        if i['date'] == date and i['time'] == time:
            lon = i['lon']
            lat = i['lat']
    if lon == '':
        lon = prev_lon
        lat = prev_lat
    prev_lon = lon
    prev_lat = lat
    list_lon.append(lon)
    list_lat.append(lat)
temp = SFrame({'lon': list_lon, 'lat': list_lat})
sen_frame = sen_frame.add_columns(temp)

In [ ]:
'''
This step check each MAC address and see if it is in the list, or
if it is on the other trip. For every MAC it store, it is going to
search for the rest which are in the same trip. Count them up and 
store the first and last detection time for each MAC.
'''
list_date = []
list_mac = []
list_start = []
list_end = []
list_count = []
length = 0
for element in sen_frame:
    date = element['date']
    mac = element['mac']
    start_time = ''
    end_time = ''
    count = 0
    index_mac = check_last_index(list_mac, mac)
    if bool(list_date) == True:
        duplicate = (list_date[index_mac] == date)
        otherTrip = (time_difference(list_start[index_mac], element['time']) > 2700)
    if mac not in list_mac or not duplicate or otherTrip:
        list_date.append(date)
        list_mac.append(mac)
        start_time = element['time']
        list_start.append(start_time)
        for item in sen_frame[length:]:
            inRange = (time_difference(start_time, item['time']) < 2700)
            if item['mac'] == mac and item['date'] == date and inRange:
                end_time = item['time']
                count+=1
        list_end.append(end_time)
        list_count.append(count)
    length+=1
        
data = SFrame({'mac':list_mac, 'date':list_date, 'start_time':list_start, 'end_time':list_end, 'count':list_count})

In [ ]:
# Going through the data for each MAC compute the distance
# between it and the adjacent MAC which is the same as it
# This step will compute the time and location difference 
# between two detection of a same MAC, and compute the 
# distance as well as the speed between two detection.
# It will create two list which stores the total distance
# by adding up every piece of location difference together
# It will also store the maximum speed for every MAC address
list_distance = []
list_max_speed = []
for item in data:
    iteration = item['count']
    MAC = item['mac']
    distance = 0
    max_speed = 0
    count = 0
    prev_lon = 0
    prev_lat = 0
    prev_time = 0
    for element in sen_frame:
        # We only need to go thourgh iteration many MACs
        if count == iteration:
            break
        else: # If MAC address are the same, then we compute the distance
            if element['mac'] == MAC:
                count+=1
                if prev_lon == 0 and prev_lat == 0 and prev_time == 0:
                    prev_lon = float(element['lon'])
                    prev_lat = float(element['lat'])
                    prev_time = element['time']
                else:
                    now_lat = float(element['lat'])
                    now_lon = float(element['lon'])
                    now_distance = find_distance(prev_lat, prev_lon, 
                                              now_lat, now_lon)
                    distance += now_distance
                    time = time_difference(prev_time, element['time'])
                    if time != 0 and max_speed < now_distance/time:
                        max_speed = now_distance/time
                    prev_lon = now_lon
                    prev_lat = now_lat
                    prev_time = element['time']
    list_distance.append(distance)
    list_max_speed.append(max_speed)
    
temp = SFrame({'distance': list_distance, 'max_speed': list_max_speed})
data = data.add_columns(temp)

In [ ]:
# Duration
list_duration = []
for element in data:
    duration = time_difference(element['start_time'], element['end_time'])
    list_duration.append(duration)
temp = SFrame({'duration':list_duration})
data = data.add_columns(temp)

# Speed
list_speed = []
for element in data:
    if element['duration'] == 0:
        list_speed.append(0)
    else:
        speed = element['distance'] / element['duration']
        list_speed.append(speed)
temp = SFrame({'avg_speed':list_speed})
data = data.add_columns(temp)

In [ ]:
feature_data = data.remove_columns(['date', 'start_time', 'end_time', 'mac'])
# Normalized the features
features = ['duration', 'count', 'distance', 'avg_speed', 'max_speed']
for f in features:
    feature_data[f] = normalize(feature_data[f])
# Make the panda frame into ndarray
input_data = feature_data.to_dataframe().values